# Model Training

### 1.1 Import Data and Required Packages

In [1]:
# Basic Imports
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

import warnings

#### Import the CSV Data as Pandas DataFrame

In [2]:
df = pd.read_csv("data/stud.csv")

#### Import the CSV Data as Pandas DataFrame

In [3]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


#### Preparing X ang y variables

In [4]:
X = df.drop('math_score', axis=1)
y = df['math_score']

In [5]:
X.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [6]:
y[:10]

0    72
1    69
2    90
3    47
4    76
5    71
6    88
7    40
8    64
9    38
Name: math_score, dtype: int64

In [7]:
# Separate dataset into train and test (avoid data leakage)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape

((800, 7), (200, 7))

In [8]:
# Create Column Transformer with 3 types of transformers
numeric_features = X.select_dtypes(exclude='object').columns
categorical_features = X.select_dtypes(include='object').columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, categorical_features),
        ("StandardScaler", numeric_transformer, numeric_features)
    ]
)

In [9]:
# X = preprocessor.fit_transform(X)

# Fit the transformer on the training data and transform the training data
X_train = preprocessor.fit_transform(X_train)

# Use the fitted transformer to transform the test data
X_test = preprocessor.transform(X_test)
X_train.shape, X_test.shape

((800, 19), (200, 19))

#### Create an Evaluate Function to give all metrics after model Training

In [10]:
def evaluate_model(true,predicted):
    """Evaluate the model performance.

    Args:
        true (np.array): True value of y. (y_test)
        predicted (np.array): Model predicted values of y.
    """
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mse)
    r2_square = r2_score(true, predicted)
    
    return mae, mse, rmse, r2_square

In [11]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decison Tree Regressor": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(),
    "CatBoost Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}

model_list = []
r2_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model
    
    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate model and test dataset
    model_train_mae, model_train_mse, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_test_mae, model_test_mse, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)
    
    print(list(models.keys())[i]) # model_name - Completed
    model_list.append(list(models.keys())[i])
    
    # Check for overfitting
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 5.3617
- Mean Absolute Error: 4.2830
- R2 Score: 0.8725
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 5.4178
- Mean Absolute Error: 4.2400
- R2 Score: 0.8794


Lasso
Model performance for Training set
- Root Mean Squared Error: 6.5925
- Mean Absolute Error: 5.2053
- R2 Score: 0.8072
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 6.5173
- Mean Absolute Error: 5.1557
- R2 Score: 0.8254


Ridge
Model performance for Training set
- Root Mean Squared Error: 5.3233
- Mean Absolute Error: 4.2650
- R2 Score: 0.8743
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 5.3904
- Mean Absolute Error: 4.2111
- R2 Score: 0.8806


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 5.7058
- Mean Absolute Error: 4.5120
- R2 Score: 0.8556
-----------------------

**Linear Regression model and Ridge model are the two model that is not overfitting and has smaller MSE.**

In [12]:
pd.DataFrame(list(zip(model_list, r2_list)), 
             columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)

,Model Name,R2_Score
2,Ridge,0.880592
0,Linear Regression,0.879374
5,Random Forest Regressor,0.855403
7,CatBoost Regressor,0.851831
8,AdaBoost Regressor,0.842364
6,XGBRegressor,0.827797
1,Lasso,0.825446
3,K-Neighbors Regressor,0.785944
4,Decison Tree Regressor,0.732471


- Ridge Regression Model beats Linear Regression Model in explaining variance.

- Linear Regression model explains **88.06%** of the variability in the target data, meaning it’s a better fit.

### Best model: Ridge Regression

In [16]:
# Initialize the Ridge regression model with default parameters
ridge_model = Ridge()

# Fit the model on the training data
ridge_model.fit(X_train, y_train)

# Predict target values for the test data
y_pred = ridge_model.predict(X_test)

# Calculate the R² score to evaluate the model's performance on the test set
score = r2_score(y_test, y_pred) * 100  # Multiply by 100 to get percentage

# Print the accuracy
print("Accuracy of the Ridge regression model is %.2f%%" % score)


Accuracy of the Ridge regression model is 88.06%


In [ ]:
# lin_model = LinearRegression(fit_intercept=True)
# lin_model = lin_model.fit(X_train, y_train)
# y_pred = lin_model.predict(X_test)
# score = r2_score(y_test, y_pred)*100
# print(" Accuracy of the model is %.2f" %score)

 Accuracy of the model is 87.94


### Plot y_pred and y_test


In [14]:
fig = px.scatter(x=y_test, y=y_pred, title="Actual value Vs Predicted value", labels={"x":"Actual", "y":"Predicted"})
fig.show()

In [ ]:
# Scatter plot for y_test vs. y_pred
fig = go.Figure()

# Scatter points
fig.add_trace(go.Scatter(
    x=y_test, y=y_pred, mode='markers', marker=dict(color='blue'),name="Data Points"
))

# Regression line (best-fit line)
m, b = np.polyfit(y_test, y_pred, 1)  # Linear regression fit for y_test vs. y_pred
fig.add_trace(go.Scatter(
    x=y_test, y=m * y_test + b,  # Line equation: y = mx + b
    mode='lines', line=dict(color='red'), name='Regression Line'
))

fig.update_layout(
    title="Regression Plot", xaxis_title="Actual", yaxis_title="Predicted"
)

fig.show()